In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

import time
import json
from pprint import pprint
from concurrent.futures import ThreadPoolExecutor, TimeoutError
import pandas as pd
import urllib.request

In [2]:
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument("--disable-extensions")
# we dont need to load images
chrome_prefs = {
      'profile.managed_default_content_settings.images': 2,
    }
chrome_options.add_argument('user-agent="Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.108 Mobile Safari/537.36"')
chrome_options.add_experimental_option('prefs', chrome_prefs)
def get_driver():
    return webdriver.Chrome(options=chrome_options)

In [3]:
# driver = get_driver()

In [4]:
# first read known app ids and names
IDTONAME = {}
with open('app_id.txt', encoding='utf-8') as f:
    for l in f.readlines():
        j = json.loads(l)
        for appid in j:
            IDTONAME[appid] = j[appid]
print(len(IDTONAME))

1000000


In [5]:
CRAWLED = set()
with open('data.jl', encoding='utf-8') as f:
    for l in f.readlines():
        try:
            j = json.loads(l)
            CRAWLED.add(j['pkg'])
        except:
            pass
print(len(CRAWLED))
# CRAWLED

622570


In [6]:
INVALID = set()
with open('invalid.txt', encoding='utf-8') as f:
    for l in f.readlines():
        INVALID.add(l.strip())
len(INVALID)

28959

In [7]:
def appendmsg(msg, filename):
    with open(filename, 'ab') as f:
        f.write(msg.encode('utf-8'))
        f.write('\n'.encode('utf-8'))

BUFSIZ = 1
BUFFER = []
def log(d):
    BUFFER.append(d)
    if len(BUFFER) > BUFSIZ:
        for i in range(BUFSIZ):
            d = BUFFER.pop(0)
            appendmsg(json.dumps(d, ensure_ascii=False), 'data.jl')

In [8]:
BASEURL = 'https://play.google.com/store/apps/details?id={}'

def get_app_detail(driver, appid):
    driver.get(BASEURL.format(appid))
    ret = {
        'name': 'unknown',
        'pkg': appid,
        'publisher': '',
        'genres': [],
        'content_ratings': [],
        'is_editor_choice': False,
        'price': 0.0,
        'inapp_details': [],
        'has_video': False,
        'num_screenshot': 0,
        'description': '',
        'rating_overall': 0.0,
        'rating_5': 0,
        'rating_4': 0,
        'rating_3': 0,
        'rating_2': 0,
        'rating_1': 0,
        'rating_count': 0,
        'additional_info': {}
    }

    #name
    ret['name'] = driver.find_element_by_class_name('AHFaub').text

    #publisher and genres
    l = driver.find_elements_by_css_selector('span.T32cc.UAO9ie')
    ret['publisher'] = l[0].text
    for i in range(1, len(l)):
        ret['genres'].append(l[i].text)

    #content rating(s)
    l = driver.find_elements_by_class_name('KmO8jd')
    for i in l:
        ret['content_ratings'].append(i.text)

    #iseditorchoice
    l = driver.find_elements_by_class_name('GcFlrd')
    if len(l) != 0:
        ret['is_editor_choice'] = True

    #price HoUVMe VdvQIe
    b = driver.find_element_by_css_selector('div.HoUVMe.VdvQIe')
    if b.text != 'Install':
        price_text = b.text.replace('$','').replace(' Buy','').strip()
        ret['price'] = float(price_text)

    #inapp details
    l = driver.find_elements_by_class_name('bSIuKf')
    for i in l:
        ret['inapp_details'].append(i.text)

    #has video
    l = driver.find_elements_by_class_name('TdqJUe')
    if len(l) != 0:
        ret['has_video'] = True

    #num screenshot
    l = driver.find_elements_by_class_name('Q4vdJd')
    ret['num_screenshot'] = len(l)

    #description
    ret['description'] = driver.find_element_by_class_name('DWPxHb').text

    #rating stuff
    ret['rating_overall'] = float(driver.find_element_by_class_name('BHMmbe').text)
    l = driver.find_elements_by_class_name('mMF0fd')

    for i in l:
        rating_score = int(i.text)
        rating_count = i.find_elements_by_tag_name('span')[1].get_attribute('title')
        rating_count = int(rating_count.replace(',',''))
        ret[f'rating_{rating_score}'] = rating_count

    rating_count = driver.find_element_by_class_name('EymY4b').text
    rating_count = rating_count.replace('total','').replace(',','')
    ret['rating_count'] = int(rating_count)

    #additional info cards
    l = driver.find_elements_by_class_name('hAyfc')
    for i in l:
        key = i.find_element_by_class_name('BgcNfc').text
        value = i.find_element_by_class_name('htlgb').text
        ret['additional_info'][key] = value

    return ret


In [9]:
# get_app_detail(driver, 'com.mojang.minecraftpe')

In [10]:
# get_app_detail(driver, 'com.instagram.android')

In [11]:
# driver.close()

In [12]:
QUEUE = []
for appid in IDTONAME:
    if appid in CRAWLED: continue
    if appid in INVALID: continue
    QUEUE.append(appid)
    
#     time.sleep(.5)
print(len(QUEUE))
QUEUE[:20]

348471


['com.rastrear.celular.top',
 'com.andromo.dev597108.app778222',
 'com.all.tube.video.downloader',
 'relationship.advice',
 'howtobe.attractive.tomen',
 'chankyaniti.quotes.quoteslovers',
 'com.kabadigital.shaketocrackdisplay',
 'com.kurnatechnologies.chanakyaniti',
 'com.tss21.gkbd.skin.google.pointpocket',
 'com.CuriosityFunGames.FartSoundsPrank2019',
 'com.andromo.dev628627.app616230',
 'com.treasurepiratee',
 'com.FishEntertainment.NewDslr.CameraHD',
 'caderno.notas.nota',
 'com.syzible.todo',
 'com.porting.indianrailway',
 'site.pangestumedia.nontonterus',
 'com.panatdamn.soundromanticsongs',
 'com.bollywood.ninties.superhit.video.songs',
 'com.lemoapps.hindisongs']

In [13]:
# driver = get_driver()

In [14]:
# print(get_app_detail(driver, QUEUE.pop(0)))

In [15]:
def task(d):
    while True:
        appid = QUEUE.pop(0)
        try:
            data = get_app_detail(d, appid)
            log(data)
            CRAWLED.add(appid)
        except:
            INVALID.add(appid)
            appendmsg(appid, 'invalid.txt')


In [16]:
NUMTHREADS = 24
executor = ThreadPoolExecutor(max_workers=NUMTHREADS)
for i in range(NUMTHREADS):
    executor.submit(task, get_driver())
    time.sleep(1)

In [39]:
print(len(QUEUE))
QUEUE[:10]

0


[]